In [13]:
import pandas as pd

import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon

In [14]:
p_taipei = pd.read_csv("points_taipei.csv")
p_taipei['index'] = range(0, len(p_taipei))

p_new_taipei = pd.read_csv("points_new_taipei.csv")
p_new_taipei['index'] = range(0, len(p_new_taipei))

def create_geometry_gdf(df):
    geometry = [Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)]) for x1, x2, y1, y2 in zip(df['x1'], df['x2'], df['y1'], df['y2'])]
    geometry_gdf = gpd.GeoDataFrame(geometry=geometry)
    geometry_gdf['index'] = range(0, len(df))
    return geometry_gdf

geometry_gdf_t = create_geometry_gdf(p_taipei)
geometry_gdf_nt = create_geometry_gdf(p_new_taipei)

In [15]:
def edu_data(file_path):
        df = pd.read_csv(file_path, header=0)
        edu1 = df[df['CATEGORY'].isin(['中高級校', '小學'])]
        edu2 = df[df['CATEGORY'].isin(['補習補校', '幼教', '大專院校'])]
        edu1_500_gdf, edu1_1000_gdf, edu1_3000_gdf = edu_count1(edu1)
        edu2_1000_gdf, edu2_2000_gdf = edu_count2(edu2)

        return edu1_500_gdf, edu1_1000_gdf, edu1_3000_gdf, edu2_1000_gdf, edu2_2000_gdf
        
def edu_count1(edu):
        # 創建一個包含醫院位置的GeoDataFrame
        edu_points1 = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(edu['POI_X'], edu['POI_Y'])])

        edu_500 = edu_points1.buffer(500)
        edu_1000 = edu_points1.buffer(1000)
        edu_3000 = edu_points1.buffer(3000)

        edu_500_gdf = gpd.GeoDataFrame(geometry=edu_500, crs=edu_points1.crs)
        edu_1000_gdf = gpd.GeoDataFrame(geometry=edu_1000, crs=edu_points1.crs)
        edu_3000_gdf = gpd.GeoDataFrame(geometry=edu_3000, crs=edu_points1.crs)

        return edu_500_gdf, edu_1000_gdf, edu_3000_gdf

def edu_count2(edu):
        # 創建一個包含醫院位置的GeoDataFrame
        edu_points1 = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(edu['POI_X'], edu['POI_Y'])])

        edu_1000 = edu_points1.buffer(1000)
        edu_2000 = edu_points1.buffer(2000)

        edu_1000_gdf = gpd.GeoDataFrame(geometry=edu_1000, crs=edu_points1.crs)
        edu_2000_gdf = gpd.GeoDataFrame(geometry=edu_2000, crs=edu_points1.crs)

        return edu_1000_gdf, edu_2000_gdf
    

def overlay_and_count(geometry_gdf, edu_gdf, prefix):
    intersection = geometry_gdf.overlay(edu_gdf, how='intersection')
    count_df = intersection.groupby('index').size().reset_index(name=prefix + '_cnt')
    return count_df

def calculate_edu_counts(geometry_gdf, edu1_500_gdf, edu1_1000_gdf, edu1_3000_gdf, edu2_1000_gdf, edu2_2000_gdf):
    edu1_500_cnt = overlay_and_count(geometry_gdf, edu1_500_gdf, 'edu1_500')
    edu1_1000_cnt = overlay_and_count(geometry_gdf, edu1_1000_gdf, 'edu1_1000')
    edu1_3000_cnt = overlay_and_count(geometry_gdf, edu1_3000_gdf, 'edu1_3000')
    edu2_1000_cnt = overlay_and_count(geometry_gdf, edu2_1000_gdf, 'edu2_1000')
    edu2_2000_cnt = overlay_and_count(geometry_gdf, edu2_2000_gdf, 'edu2_2000')
    return edu1_500_cnt, edu1_1000_cnt, edu1_3000_cnt, edu2_1000_cnt, edu2_2000_cnt



def calculate_weighted_score(df, *args):
    # 進行多個 merge 操作
    for arg in args:
        df = df.merge(arg, on='index', how='left')
    
    # 將 NaN 值填充為 0
    df = df.fillna(0)
    
    # 計算加權分數
    df['學校加權分數'] = (
        df['edu1_500_cnt'] * 0.4 +
        df['edu1_1000_cnt'] * 0.7 +
        df['edu1_3000_cnt'] * 1.2 +
        df['edu2_1000_cnt'] * 0.5 +
        df['edu2_2000_cnt'] * 0.3
    )
    
    return df


In [16]:
for i in range(2019,2025):
    edu1_500_gdf, edu1_1000_gdf, edu1_3000_gdf, edu2_1000_gdf, edu2_2000_gdf = edu_data(f"data_yaer\{i}_edu.csv")
    
    # 呼叫函式
    edu1_500_cnt_t, edu1_1000_cnt_t, edu1_3000_cnt_t, edu2_1000_cnt_t, edu2_2000_cnt_t = calculate_edu_counts(geometry_gdf_t, edu1_500_gdf, edu1_1000_gdf, edu1_3000_gdf, edu2_1000_gdf, edu2_2000_gdf)
    edu1_500_cnt_nt, edu1_1000_cnt_nt, edu1_3000_cnt_nt, edu2_1000_cnt_nt, edu2_2000_cnt_nt = calculate_edu_counts(geometry_gdf_nt, edu1_500_gdf, edu1_1000_gdf, edu1_3000_gdf, edu2_1000_gdf, edu2_2000_gdf)

    # 呼叫函式
    result_t = calculate_weighted_score(p_taipei, edu1_500_cnt_t, edu1_1000_cnt_t, edu1_3000_cnt_t, edu2_1000_cnt_t, edu2_2000_cnt_t)
    result_nt = calculate_weighted_score(p_new_taipei, edu1_500_cnt_nt, edu1_1000_cnt_nt, edu1_3000_cnt_nt, edu2_1000_cnt_nt, edu2_2000_cnt_nt)
    # 寫入csv
    result_t.to_csv(f"year_score/{i}_taipei_edu.csv", index=False)
    result_nt.to_csv(f"year_score/{i}_new_taipei_edu.csv", index=False)
    print(i)

2019
2020
2021
2022
2023
2024
